# Quick introduction to PyBaMM

Within this project, the modelling functionalities of PyBaMM will be used.
**PyBaMM** (Python Battery Mathematical Modelling) is an open-source battery simulation package written in Python. Amongst others, PyBaMM allows for simple battery simulations employing several types of electrochemical battery models, the DFN (Doyle-Fuller-Newman, or P2D) model, or its simplifications, the SPM (single particle model) and SPMe (single particle model with electrolyte). These basic models build up on systems of coupled differential equations, describing physical phenomena that happen inside a battery during operation (i.e. Li-ion transport, diffusion through electrolyte and active materials, charge-transfer reactions, etc.) – the details of which are outside the scope of this project.

PyBaMM is a python package and can easily be installed using pip or conda, i.e.:
 
```pip install pybamm```

Note that PyBaMM currently requires python 3.8 or 3.9 and does not run on python 3.10.

Further notes on installation and documentation can be found here: 
- https://www.pybamm.org/ 
- Source code and tutorials: https://github.com/pybamm-team/PyBaMM
- Documentation: https://pybamm.readthedocs.io/en/latest/index.html 


## Getting started
This section gives a brief introduction on how to start a simple simulation within PyBaMM, partially based on some of the available introductory tutorials. This is far from being complete. Many more options and customizations are available, most of which are outside the scope of this project. Details can be found in the PyBaMM documentation.

In [ ]:
import pybamm

#### Selecting a model
The first step is to select a model. Within this project we will DFN, SPMe and SPM, of which DFN is the most detailled and thus also computationally most expensive, followed by SPMe and SPM.

In [ ]:
# select & load model
#model = pybamm.lithium_ion.SPM()
model = pybamm.lithium_ion.SPMe()
#model = pybamm.lithium_ion.DFN()

Each model requires a different set of variables. To get a full list of the available variables type:

In [ ]:
model.variable_names()

Note that there are two variables for most quantities. This is because PyBaMM utilises both dimensionless and dimensional variables for these quantities. As a rule, the dimensionless variables have no units in their name and the dimensional variables have units in their name. If in doubt, it is recommend using the dimensional variable with units.


As the list of parameters is VERY long, it makes sense to search for parameters using keywords (e.g. "electrolyte", "voltage", "capacity"). 

In [ ]:
model.variables.search("potential")

#### Select a parameter set
As a next step, we have to specify which parameters (i.e. which specific battery type) we want to simulate. Several different parametersets are available within PyBaMM (for references see section at the end of the notebook). Here we will start with the Chen2020 parameterset, that is based on an LG NMC811/Graphite cell.

In [ ]:
list(pybamm.parameter_sets)

In [ ]:
# select parameterset
parameter_values = pybamm.ParameterValues("Chen2020") # Chen2020 is NMC811/Graphite cell

All parameters are stored in a dictionary, and can easily be viewed, searched and modified:

In [ ]:
parameter_values

In [ ]:
parameter_values=pybamm.ParameterValues("Marquis2019")
# search for parameters
parameter_values.search("heat")

In [ ]:
# change parameter values
parameter_values["Ambient temperature [K]"] = 288.15

For more info on parameters see https://colab.research.google.com/github/pybamm-team/PyBaMM/blob/main/examples/notebooks/Getting%20Started/Tutorial%204%20-%20Setting%20parameter%20values.ipynb

#### Running the simulation

In [ ]:
# create a simulation (used to process and solve the model)
sim = pybamm.Simulation(model, parameter_values=parameter_values)

In [ ]:
# solve the model
sim.solve(t_eval=[0,3600]) # if no experiment is specified, a time interval (s) for the simulation has to be specified

### Working with the solution

#### Basic plotting
There are several inbuilt plotting functionalities. Standard plot using standard variables:

In [ ]:
sim.plot()

Customize the plot by selecting output variables (select from model variable list described above):

In [ ]:
output_variables = ["Terminal voltage [V]"]
sim.plot(output_variables=output_variables)

In [ ]:
output_variables = ["Electrolyte concentration [mol.m-3]", "Terminal voltage [V]"]
sim.plot(output_variables=output_variables)

In [ ]:
sim.plot([["Electrode current density", "Electrolyte current density"], "Terminal voltage [V]"])

To access the solution variables, a solution object has to be created:

In [ ]:
solution = sim.solution

Variables can then be accessed and further processed:

In [ ]:
t = solution["Time [s]"]
V = solution["Terminal voltage [V]"]

In [ ]:
t.entries

In [ ]:
V.entries

The solution object or the data can then be saved (and loaded for later use):

In [ ]:
#solution.save("sol.pkl")
#solution.save_data("sol_data.pkl", ["Current [A]", "Terminal voltage [V]"],  to_format="csv")

#solution1 = pybamm.load("sol.pkl")

### Defining experiments
Without any further specifications, the experiment performed is the one specified in the input parameter set. For simulatin different types of experiments, the **experiment class** can be used.

The experiment class is a handy tool for simulating different types of battery experiments. This can be done by passing on a set of instructions on how to cycle the battery, e.g.,:

In [ ]:
experiment = pybamm.Experiment(
    [
        ("Discharge at C/10 for 10 hours or until 3.3 V",
        "Rest for 1 hour",
        "Charge at 1 A until 4.1 V",
        "Hold at 4.1 V until 50 mA",
        "Rest for 1 hour"),
    ] * 3
)

A cycle is defined by a tuple of operating instructions. In this case, the experiment consists of a cycle of constant current C/10 discharge, a one hour rest, a constant current (1 A) constant voltage (4.1 V) and another one hour rest, all of it repeated three times (notice the * 3).

We can then create a new simulation by passing on our defined experiment using the experiment keyword argument (in addition to the model and parameter values that we reuse from above):

In [ ]:
sim = pybamm.Simulation(model, parameter_values=parameter_values, experiment=experiment)

In [ ]:
sim.solve() # note that here we don't have to specify evaluation time; this is included in the experiment
sim.plot()

For more details and further examples on experiment definitions see https://colab.research.google.com/github/pybamm-team/PyBaMM/blob/main/examples/notebooks/Getting%20Started/Tutorial%205%20-%20Run%20experiments.ipynb#scrollTo=e3ZjqOoQF3I4

Another useful parameter to include in simulations is the initial state-of-charge. Commonly, this is taken from the initial conditions given in the parameter set, but can be specified within 0 and 1:

In [ ]:
sim1 = pybamm.Simulation(model, parameter_values=parameter_values, experiment=experiment)
sim1.solve(initial_soc=0.5)
sim1.plot()

## Examples

### 1. Adding a thermal model
Different options can be passed on to the model (as a dictionary), e.g. thermal models. Thermal models within PyBaMM are “isothermal” (default), “lumped”, “x-lumped”, or “x-full”.

Note that a parameterset containing the relevant thermal parameters for this optional model has to be chosen (or relevant parameters have to be added to the parameterset). For more info on what type of options can be used, see https://pybamm.readthedocs.io/en/latest/source/api/models/base_models/base_battery_model.html

In [ ]:
#options = {"thermal": "x-full"}
options = {"thermal": "lumped"}
model_thermal = pybamm.lithium_ion.SPMe(options=options)
parameter_values_thermal = pybamm.ParameterValues("Marquis2019")

In [ ]:
sim_thermal = pybamm.Simulation(model_thermal, parameter_values=parameter_values_thermal)
sim_thermal.solve([0, 3600])
sim_thermal.plot([
    "Terminal voltage [V]",
    "X-averaged cell temperature [K]",
    "Cell temperature [K]",
])

### 2. Rate capability
Calculate the capacity achieved at different C-rates.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
model = pybamm.lithium_ion.SPMe()
parameter_values = pybamm.ParameterValues("Chen2020")

C_rates = np.linspace(0.05, 5, 10)
capacities = np.zeros_like(C_rates)
currents = np.zeros_like(C_rates)
voltage_av = np.zeros_like(C_rates)

In [ ]:
for i, C_rate in enumerate(C_rates):
    experiment = pybamm.Experiment(
        ["Discharge at {:.4f}C until 3.2V".format(C_rate)],
        period="{:.4f} seconds".format(10 / C_rate),
    )
    sim = pybamm.Simulation(model, experiment=experiment, solver=pybamm.CasadiSolver())
    sim.solve()

    time = sim.solution["Time [s]"].entries
    capacity = sim.solution["Discharge capacity [A.h]"]
    current = sim.solution["Current [A]"]
    voltage = sim.solution["Terminal voltage [V]"]

    capacities[i] = capacity(time[-1])
    currents[i] = current(time[-1])
    voltage_av[i] = np.mean(voltage(time))

In [1]:
plt.figure(1)
plt.scatter(C_rates, capacities)
plt.xlabel('C-rate')
plt.ylabel('Capacity [Ah]')

plt.figure(2)
plt.scatter(currents * voltage_av, capacities * voltage_av)
plt.xlabel('Power [W]')
plt.ylabel('Energy [Wh]')

plt.show()

NameError: name 'plt' is not defined

## References
Get references for papers used within the PyBaMM code:

In [ ]:
pybamm.print_citations()